# What is unit testing?

# Why do we do unit testing?

# How do we do unit testing?

In [ ]:
def adds_x(num, x):
    return(num + x)

In [ ]:
def adds_x():
    assert adds_x(3, 1) == 4

In [ ]:
def test_adds_one():
    result = adds_x(3, 1)
    expect = 4
    assert result == expect, f"adds_x should return 4 when given 3 and 1, but instead returned {result}"

In [ ]:
@pytest.mark.parametrize("x", 3)
def test_adds_one(x):
    result = adds_x(2, x)
    expect = 2+x
    assert result == expect, f"adds_x should return {expect} when given 2 and {x}, but instead returned {result}"

In [ ]:
@pytest.mark.parametrize("num, x", [(3, 1), (0, -1), (-5, 2)])
def test_adds_one(num, x):
    result = adds_x(num, x)
    expect = x+1
    assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"

# How can I make unit testing less of a slog?

Property-based testing (e.g. Hypothesis in Python)